# PACKAGES LOADING

In [1]:
!pip install pymupdf
!pip install sentence-transformers
!pip install langchain
!pip install transformers
!pip install langchain_community
!pip install torch
!pip install psutil
!pip install chromadb



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [2]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.3 MB/s eta 0:00:00


In [20]:
!pip install huggingface_hub

In [22]:
!pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 MB 6.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.3 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.8-cp311-cp311-linux_x86_64.whl size=5959640 sha256=c032f7fa930dc2dbb49bcf73ef99aa8755bd90943dc0947152155a5697065fb4
  Stored in directory: /root/.cache/pip/wheels/c0/03/66/eb3810eafd55d921b2be32896d1f44313996982360663aa80b
Successfully built llama-cpp-python


In [25]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma
from transformers import pipeline, AutoTokenizer

import os
import pickle
import torch
import psutil
import chromadb
from concurrent.futures import ThreadPoolExecutor, as_completed
import re
import textwrap
from huggingface_hub import hf_hub_download
from llama_cpp import Llama


In [26]:
from google.colab import userdata
token_hf = userdata.get('HF_Token')

In [27]:
from huggingface_hub import hf_hub_download, login
login()

In [28]:
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/MyDrive/PGP_AI_ML_GREAT_LEARNING/05_NLP With Gen AI/06_Final Project/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/PGP_AI_ML_GREAT_LEARNING/05_NLP With Gen AI/06_Final Project


In [29]:
!ls

'=1.26.0'	        Medical_Assistant_v1_1.ipynb	  Reference
 Archives	        medical_diagnosis_manual.pdf	  Vector_Directory
 chunk_checkpoint.txt   NLP_Medi_Assist.ipynb
 Chunk_Directory        Project_Template_Notebook.ipynb


# LLM

In [30]:
model_name_llma_path = "TheBloke/Llama-2-13B-chat-GGUF"
model_LLMA_BaseName = "llama-2-13b-chat.Q5_K_M.gguf"

In [31]:
model_llma_path = hf_hub_download(repo_id = model_name_llma_path,
                                  filename=model_LLMA_BaseName, token = token_hf)

llama-2-13b-chat.Q5_K_M.gguf:   0%|          | 0.00/9.23G [00:00<?, ?B/s]

In [33]:
llama_model = Llama(model_path=model_llma_path,
                    n_threads=2, n_batch=512, n_ctx=4096)

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGUF/snapshots/4458acc949de0a9914c3eab623904d4fe999050a/llama-2-13b-chat.Q5_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_

In [34]:
def response(query,max_tokens=128,temperature=0,top_p=0.95,top_k=50):
    model_output = llama_model(
      prompt=query,
      max_tokens=max_tokens,
      temperature=temperature,
      top_p=top_p,
      top_k=top_k
    )

    return model_output['choices'][0]['text']

## without changing the hyperparameter

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [ ]:
query_01 = 'What is the protocol for managing sepsis in a critical care unit?'
result_query_1=response(query_01)
result_query_1

'\nSepsis is a life-threatening condition that can arise from an infection, and it is a leading cause of death in hospitals. The management of sepsis in a critical care unit (CCU) requires a systematic approach that includes early recognition, prompt treatment, and ongoing monitoring. Here is a general protocol for managing sepsis in a CCU:\n\n1. Early recognition:\n\t* Use a standardized screening tool, such as the Systemic Inflammatory Response Syndrome (SIRS) criteria or the Sepsis-3 definition'

Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [ ]:
query_2 = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
result_query_2=response(query_2)
result_query_2

Llama.generate: prefix-match hit


'\n\nAnswer:\n\nAppendicitis is a medical emergency that requires prompt treatment. The most common symptoms of appendicitis include:\n\n1. Severe pain in the abdomen, usually starting near the belly button and then moving to the lower right side of the abdomen.\n2. Nausea and vomiting.\n3. Loss of appetite.\n4. Fever.\n5. Abdominal tenderness and guarding (muscle tension).\n6. Abdominal swelling.\n7. Diarrhea or constipation'

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [ ]:
query_3 = 'What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?'
result_query_3=response(query_3)
result_query_3

Llama.generate: prefix-match hit


'\nSudden patchy hair loss, also known as alopecia areata, can be caused by a variety of factors. Here are some effective treatments and solutions for addressing this condition:\n\n1. Corticosteroid injections: These injections can help suppress the immune system and promote hair growth. They are usually given every 4-6 weeks and can be effective in promoting hair regrowth.\n2. Topical corticosteroids: Over-the-counter or prescription topical corticosteroids can be applied directly to the affected area to reduce inflamm'

### Query:4 What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [ ]:
query_4 = 'What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?'
result_query_4 = response(query_4)
result_query_4

Llama.generate: prefix-match hit


"\nThere are several treatments that may be recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function. The specific treatment plan will depend on the location and severity of the injury, as well as the individual's overall health and medical history. Some common treatments for brain injuries include:\n\n1. Medications: To manage symptoms such as pain, inflammation, and anxiety.\n2. Rehabilitation therapy: To help regain lost function and improve cognitive, physical, and emotional abilities."

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [ ]:
query_5 = 'What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?'
result_query_5 = response(query_5)
result_query_5

Llama.generate: prefix-match hit


'\n\nA person who has fractured their leg during a hiking trip requires prompt medical attention to ensure proper healing and prevent complications. Here are some necessary precautions and treatment steps:\n\n1. Stop all activity: The first step is to stop all activity and rest the affected leg immediately. This will help prevent further damage and reduce the risk of complications.\n2. Seek medical attention: It is essential to seek medical attention as soon as possible, especially if the fracture is displaced or if there are any signs of nerve or blood vessel damage. A healthcare professional can'

## Query response Hyperparameter Tuned

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [ ]:
query_1 = 'What is the protocol for managing sepsis in a critical care unit?'
result_tuned_Query_1 = response(query_1,max_tokens=228,temperature=0.87, top_p=0.87,top_k=49)
result_tuned_Query_1


Llama.generate: prefix-match hit


"\nThe management of sepsis in a critical care unit involves a multidisciplinary approach that includes early recognition, prompt administration of antibiotics and supportive care, and careful monitoring of the patient's clinical status. The following is a general protocol for managing sepsis in a critical care unit:\n\n1. Early recognition:\n\t* Use of a screening tool such as the Systemic Inflammatory Response Syndrome (SIRS) criteria or the Sepsis-3 definition to identify patients with suspected sepsis.\n\t* Assessment of vital signs, including temperature, blood pressure, tachycardia, and tachypnea.\n\t* Evaluation of organ dysfunction, such as changes in mentation, respiratory rate, or peripheral oxygen saturation.\n2. Initial resuscitation:\n\t* Administer IV fluids and vasopressors as needed to maintain mean arterial pressure (MAP) ≥65 mmHg and serum lactate"

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [ ]:
query_02 = 'What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?'
result_tuned_Query_2 = response(query_02, max_tokens=256, temperature=0.5, top_p=0.75, top_k=45)
result_tuned_Query_2

Llama.generate: prefix-match hit


'\n\nAnswer: Appendicitis is a medical emergency that requires prompt treatment. The most common symptoms of appendicitis include:\n\n1. Severe pain in the abdomen, usually starting near the belly button and then moving to the lower right side of the abdomen.\n2. Nausea and vomiting.\n3. Loss of appetite and fever.\n4. Abdominal tenderness and guarding (muscle tension) when the abdomen is touched.\n5. Abdominal swelling and rigidity.\n6. Diarrhea or constipation.\n7. Inability to pass gas.\n\nIf you suspect that you or someone else may have appendicitis, it is essential to seek medical attention immediately. Appendicitis can be cured via surgery, but not via medicine. The surgical procedure used to treat appendicitis is called an appendectomy, which involves removing the inflamed appendix.\n\nThere are two types of appendectomies: open and laparoscopic. Open appendectomy is the traditional method, where a single incision is made in the lower right ab'

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [ ]:
query_03 = 'What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?'
result_tuned_query_3 = response(query_03,max_tokens=275,temperature=0.75,top_p=.8,top_k=47)
result_tuned_query_3

Llama.generate: prefix-match hit


"\nHair loss, especially sudden patchy hair loss, can be a distressing condition that affects both men and women. Localized bald spots on the scalp can be caused by various factors, and it's essential to identify the underlying cause to determine the most effective treatment. Here are some possible causes of sudden patchy hair loss and their treatments:\n\n1. Telogen effluvium: This is a common condition that occurs when there is an increase in the number of hair follicles that stop growing and enter the resting phase (telogen phase). This can cause sudden hair loss, especially on the scalp. Treatment options include medications like minoxidil (Rogaine) and finasteride (Propecia), as well as addressing any underlying stressors or hormonal imbalances.\n2. Alopecia areata: This is an autoimmune condition that causes hair loss in patches. It can occur at any age and affects both men and women. Treatment options include topical corticosteroids, immunotherapy, and systemic medications like 

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [ ]:
query_04= 'What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?'
result_tuned_query_4 = response(query_04,max_tokens=156, temperature=0.65, top_p=0.85, top_k=50)
result_tuned_query_4

Llama.generate: prefix-match hit


'\nThere are several treatments that may be recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function. The specific treatment plan will depend on the location and severity of the injury, as well as the individual needs and circumstances of the patient. Some possible treatments include:\n1. Medications: Various medications may be prescribed to help manage symptoms such as pain, anxiety, or depression. These may include analgesics, anti-anxiety drugs, or antidepressants.\n2. Rehabilitation therapy: Physical, occupational, and speech therapy may be necessary to help the patient regain lost function and improve their'

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [ ]:
query_05 = 'What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?'
result_tuned_query_5 = response(query_05,max_tokens=256, temperature=0.95, top_p=0.85, top_k=50)
result_tuned_query_5

Llama.generate: prefix-match hit


'\n\n\nA person who has fractured their leg while hiking requires immediate medical attention to prevent further complications and ensure proper healing. The necessary precautions and treatment steps for such a person are as follows:\n\n1. Stop all activities: The first step is to stop all activities and rest the affected leg immediately. This will help prevent further damage or complications.\n\n2. Apply RICE method: RICE stands for Rest, Ice, Compression, and Elevation. Applying the RICE method will help reduce pain, swelling, and inflammation. Rest the affected leg, apply ice packs to reduce swelling, compress the area with an elastic bandage, and elevate the leg above heart level.\n\n3. Seek medical attention: It is essential to seek medical attention as soon as possible, especially if there are signs of infection, such as redness, swelling, or pus. A healthcare professional can assess the severity of the fracture and provide appropriate treatment.\n\n4. Immobilize the leg: To immo

## BLUE SCORE CALCULATION


In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

response_std_hyperparameter = [result_query_1,result_query_2, result_query_3, result_query_4, result_query_5]
response_tuned_hyperparameter = [result_tuned_Query_1, result_tuned_Query_2, result_tuned_query_3, result_tuned_query_4, result_tuned_query_5]

smmoth_Object = SmoothingFunction()
bleu_scores=[]
for indx,(reference, tuned_response) in enumerate(zip(response_std_hyperparameter, response_tuned_hyperparameter)):
  bleu_score = sentence_bleu([reference],tuned_response,smoothing_function=smmoth_Object.method1)
  bleu_scores.append(bleu_score)
  print(f'Blue Score for the index: {indx+1} is {bleu_score:.4f}')


Blue Score for the index: 1 is 0.4605
Blue Score for the index: 2 is 0.4266
Blue Score for the index: 3 is 0.3173
Blue Score for the index: 4 is 0.7071
Blue Score for the index: 5 is 0.5053


In [ ]:
default_Param = 'max_tokens=128,temperature=0,top_p=0.95,top_k=50'

tuned_Query_1 = 'max_tokens=228,temperature=0.87, top_p=0.87,top_k=49'
tuned_Query_2 = "max_tokens=256, temperature=0.5, top_p=0.75, top_k=45"
tuned_Query_3 = "max_tokens=275,temperature=0.75,top_p=.8,top_k=47"
tuned_Query_4 = "max_tokens=156, temperature=0.65, top_p=0.85, top_k=50"
tuned_Query_5 = "max_tokens=256, temperature=0.95, top_p=0.85, top_k=50"


In [ ]:
result =[{"Default":default_Param, "Tuned":tuned_Query_1, "BLEU SCORE": bleu_scores[0]},
         {"Default":default_Param, "Tuned":tuned_Query_2, "BLEU SCORE": bleu_scores[1]},
         {"Default":default_Param, "Tuned":tuned_Query_3, "BLEU SCORE": bleu_scores[2]},
         {"Default":default_Param, "Tuned":tuned_Query_4, "BLEU SCORE": bleu_scores[3]},
         {"Default":default_Param, "Tuned":tuned_Query_5, "BLEU SCORE": bleu_scores[4]},
         ]

In [ ]:
df_BLEU_SCORE = pd.DataFrame(result)

In [ ]:
df_BLEU_SCORE

,Default,Tuned,BLEU SCORE
0,"max_tokens=128,temperature=0,top_p=0.95,top_k=50","max_tokens=228,temperature=0.87, top_p=0.87,to...",0.460502
1,"max_tokens=128,temperature=0,top_p=0.95,top_k=50","max_tokens=256, temperature=0.5, top_p=0.75, t...",0.426552
2,"max_tokens=128,temperature=0,top_p=0.95,top_k=50","max_tokens=275,temperature=0.75,top_p=.8,top_k=47",0.317341
3,"max_tokens=128,temperature=0,top_p=0.95,top_k=50","max_tokens=156, temperature=0.65, top_p=0.85, ...",0.707140
4,"max_tokens=128,temperature=0,top_p=0.95,top_k=50","max_tokens=256, temperature=0.95, top_p=0.85, ...",0.505343


# RAG

In [36]:
print("Retrivel Auggumented Generation")

# Load the Merck manual and display the first 5 pages

In [37]:
def Load_Display_MerckManaul(merck_manual_path, pages_to_print):
  merck_manual = PyMuPDFLoader(merck_manual_path).load()
  print(f'Total pages in the Merck Manual {len(merck_manual)}')

  for page in merck_manual[:pages_to_print]:
    print(f'Page Number: {page.metadata["page"]} \n')
    print(page.page_content)

  return merck_manual




In [38]:
merck_manual=Load_Display_MerckManaul('medical_diagnosis_manual.pdf',10)

In [39]:
merck_manual[150]

Document(metadata={'producer': 'pdf-lib (https://github.com/Hopding/pdf-lib)', 'creator': 'Atop CHM to PDF Converter', 'creationdate': '2012-06-15T05:44:40+00:00', 'source': 'medical_diagnosis_manual.pdf', 'file_path': 'medical_diagnosis_manual.pdf', 'total_pages': 4114, 'format': 'PDF 1.7', 'title': 'The Merck Manual of Diagnosis & Therapy, 19th Edition', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2025-03-18T08:53:05+00:00', 'trapped': '', 'modDate': 'D:20250318085305Z', 'creationDate': 'D:20120615054440Z', 'page': 150}, page_content='CO2 is also produced by bacterial metabolism and generated in the reaction of HCO3- and H+. H+ may\ncome from gastric HCl or from fatty acids released during digestion of fats—the latter sometimes\nproduces several hundred mEq of H+. The acid products released by bacterial fermentation of\nunabsorbed carbohydrates in the colon may also react with HCO3- to produce CO2. Although bloating\nmay occasionally occur, the rapid diffusion of CO2 int

# Chunk the documents

In [40]:
def Chunk_Merck_Manual(documents, chunk_directory='Chunk_Directory',chunk_size=512,chunk_overlap =20,batch_size=50):
  checkpoint_file = 'chunk_checkpoint.txt'
  last_batch=-1
  if os.path.exists(checkpoint_file):
    with open(checkpoint_file, 'r') as f:
      last_batch = int(f.read().strip())
    print(f'Resuming chunking from batch {last_batch}')

  # Tiktoken- bases splitter
  splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
      chunk_size = chunk_size,
      encoding_name = 'cl100k_base', chunk_overlap = chunk_overlap
      )
  chunked_docs = splitter.split_documents(documents)
  total_chunks = len(chunked_docs)

  chunk_id = last_batch + 1
  for indx in range(chunk_id*batch_size, total_chunks, batch_size):
    batch = chunked_docs[indx:indx + batch_size]
    with open(f'{chunk_directory}/chunk_{chunk_id}.pkl', 'wb') as f:
      pickle.dump(batch, f)
    print(f'Chunk saved batch {len(batch)} chunks to {chunk_directory}/chunks_{chunk_id}.pkl')
    chunk_id += 1

    with open(checkpoint_file, 'w') as f:
      f.write(str(chunk_id -1))

  return (total_chunks + batch_size-1) // batch_size








In [41]:
chunk_files = Chunk_Merck_Manual(merck_manual)

In [42]:
def batch_embedding(batch, embedding_function, batch_id):
  print(f'Batch {batch_id} started processing')
  b_texts = [doc.page_content for doc in batch]
  b_embeddings = embedding_function.embed_documents(b_texts)

  # for b_doc, b_embed in zip(batch,b_embeddings):
  #   b_doc.embedding = b_embed

  print(f'Batch {batch_id} completed processing')

  return b_texts,b_embeddings

In [43]:
def Vector_Creation(num_chunk_files, persist_dire='Vector_Directory', USE_GPU=False, chunk_dir = 'Chunk_Directory',):
  embedding_function = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')
  checkpoint_file = f'{persist_dire}/embedding_vector_checkpoint.txt'
  temp_checkpoint_file = f'{persist_dire}/embedding_vector_checkpoint_temp.txt'
  last_processed_batch = -1

  if os.path.exists(checkpoint_file):
    with open(checkpoint_file, 'r') as f:
      last_processed_batch = int(f.read().strip())
    vector_store = Chroma(persist_directory=persist_dire,
                          embedding_function = embedding_function)
    print(f'Resuming embedding vector from batch {last_processed_batch}')
  else:
    vector_store = Chroma(embedding_function = embedding_function,
                          persist_directory=persist_dire)

    print('Initialized new Chroma Vector store DB')



  if USE_GPU and torch.cuda.is_available():
    for indx in range(last_processed_batch + 1, num_chunk_files):
      chunk_file = f'{chunk_dir}/chunk_{indx}.pkl'
      if not os.path.exists(chunk_file):
        print(f'chunk file {chunk_file} is missing')
        break

      # loading the Chunk pickle file
      with open(chunk_file, 'rb') as f:
        batch = pickle.load(f)

      # wirtting the temp file that the batch {indx} is progress
      with open(temp_checkpoint_file, 'w') as f:
        f.write(str(indx))

      ## Add to chroma DB and persists
      vector_store.add_documents(documents=batch)
      vector_store.persist()

      ## Update the main checkpoint and clean up temp
      with open(checkpoint_file, 'w') as f:
        f.write(str(indx))
      if os.path.exists(temp_checkpoint_file):
        os.remove(temp_checkpoint_file)

  else:
    batch_queue = []
    for indx in range(last_processed_batch+1, num_chunk_files):
      chunk_file = f'{chunk_dir}/chunk_{indx}.pkl'
      if not os.path.exists(chunk_file):
        print(f'chunk file {chunk_file} is missing')
        break
      # loading the Chunk pickle file
      with open(chunk_file, 'rb') as f:
        batch = pickle.load(f)
      batch_queue.append((indx,batch))

    with ThreadPoolExecutor(max_workers=2) as executor:
      future_to_process= {}
      for mt, batch in batch_queue:
        print(f'Processing the  embedding for batch {mt}')
        future = executor.submit(batch_embedding, batch, embedding_function,mt)
        future_to_process[future] = mt

      for future in as_completed(future_to_process):
        batch_id = future_to_process[future]

        try:
          pass
          b_texts,b_embeddings = future.result()
          print(f'Chroma addition for the batch {batch_id} in progress')
          metadatas = [{'batch_id': batch_id, "Chunk_idx":i }
                       for i in range(len(b_texts))]

          with open(temp_checkpoint_file, 'w') as f:
            f.write(str(batch_id))
          vector_store.add_texts(texts=b_texts, embeddings=b_embeddings,
                                 metadatas=metadatas)
          print(f'Batch {batch_id} was processed in the memory')
          if batch_id % 5 == 0:
            vector_store.persist()
            print(f" presisted the chroma db at batch{batch_id}")
          print(f'Batch {batch_id} is added to chroma db')



          with open(checkpoint_file, 'w') as f:
            f.write(str(batch_id))
          if os.path.exists(temp_checkpoint_file):
            os.remove(temp_checkpoint_file)
        except Exception as ex:
          print(f'Exception Occured while processing the batch {batch_id}, \n exception {ex}')

        if psutil.virtual_memory().percent >90:
          print("Memory utilization is high")
          break



  try:
    vector_store.persist()
    print('Final persistance completed')
  except Exception as Ex:
    print(f'Exception while persist,{Ex}')

  # uuid_folder_path = None
  # for item_f in os.listdir(persist_dire):
  #   if os.path.isdir(os.path.join(persist_dire, item_f) and item_f!= '__pycache__'):
  #     uuid_folder_path = item_f
  #     break
  # if uuid_folder_path:
  #   path_sqlite = os.path.join(persist_dire, uuid_folder_path, 'chroma.sqlite3')
  #   print(f'Chroma database saved at {path_sqlite}')
  # else:
  #   print('No uuid folder found in persist directory')




  return vector_store








In [64]:
vector_store = Vector_Creation(chunk_files)

## RAG RESPONSE

In [80]:
def Retrival_Response_From_RAG(query, vector_store, k=3,similarity_threshold=0.0):
  try:
    q_results = vector_store.similarity_search_with_score(query=query, k=k)
    chunk_retrived = []
    for res,q_score in q_results:
      if q_score >= similarity_threshold:
        chunk_information = {"content": res.page_content,
                           "metadata": res.metadata if res.metadata else {},
                             "similarity_score": q_score}

        chunk_retrived.append(chunk_information)
    print(f'Similarity Scores for the input query: {[chunk["similarity_score"] for chunk in chunk_retrived]}')
    print(f'Chunks Retrived {len(chunk_retrived)} for the query \n {query}')

    if not chunk_retrived:
      return "No information retrived from the mode for the query asked"
    return chunk_retrived


  except Exception as ex:
    print(f'OOPS!!! uable to retrive answer for your query. \n Exception: {ex}')
    return []

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [69]:
rag_Query_01 = 'What is the protocol for managing sepsis in a critical care unit?'
result_rag_query_01 = Retrival_Response_From_RAG(rag_Query_01,vector_store)

In [70]:
result_rag_query_01

[{'content': "16 - Critical Care Medicine\nChapter 222. Approach to the Critically Ill Patient\nIntroduction\nCritical care medicine specializes in caring for the most seriously ill patients. These patients are best\ntreated in an ICU staffed by experienced personnel. Some hospitals maintain separate units for special\npopulations (eg, cardiac, surgical, neurologic, pediatric, or neonatal patients). ICUs have a high\nnurse:patient ratio to provide the necessary high intensity of service, including treatment and monitoring\nof physiologic parameters.\nSupportive care for the ICU patient includes provision of adequate nutrition (see p. 21) and prevention of\ninfection, stress ulcers and gastritis (see p. 131), and pulmonary embolism (see p. 1920). Because 15 to\n25% of patients admitted to ICUs die there, physicians should know how to minimize suffering and help\ndying patients maintain dignity (see p. 3480).\nPatient Monitoring and Testing\nSome monitoring is manual (ie, by direct obser

### Query 2 What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [71]:
rag_Query_02 = 'What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?'
result_rag_query_02 = Retrival_Response_From_RAG(rag_Query_02,vector_store)
result_rag_query_02

[{'content': "Appendicitis\nAppendicitis is acute inflammation of the vermiform appendix, typically resulting in abdominal\npain, anorexia, and abdominal tenderness. Diagnosis is clinical, often supplemented by CT or\nultrasound. Treatment is surgical removal.\nIn the US, acute appendicitis is the most common cause of acute abdominal pain requiring surgery. Over\n5% of the population develops appendicitis at some point. It most commonly occurs in the teens and 20s\nbut may occur at any age.\nOther conditions affecting the appendix include carcinoids, cancer, villous adenomas, and diverticula. The\nappendix may also be affected by Crohn's disease or ulcerative colitis with pancolitis.\nThe Merck Manual of Diagnosis & Therapy, 19th Edition\nChapter 11. Acute Abdomen & Surgical Gastroenterology\n163\njpkarthikk@outlook.com\nJS6E3YI1Z7\nThis file is meant for personal use by jpkarthikk@outlook.com only.\nSharing or publishing the contents in part or full is liable for legal action.",
  'me

### Query 3 What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [72]:
rag_Query_03 = 'What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?'
result_rag_query_03 = Retrival_Response_From_RAG(rag_Query_03,vector_store)
result_rag_query_03

[{'content': 'Nonscarring alopecia results from processes that reduce or slow hair growth without irreparably\ndamaging the hair follicle. Disorders that primarily affect the hair shaft also are considered nonscarring\nalopecia.\nEtiology\nThe alopecias comprise a large group of disorders with multiple and varying etiologies (see Table 86-1).\nThe most common cause of alopecia is\n• Androgenetic alopecia (male-pattern or female-pattern hair loss)\nAndrogenetic alopecia is an androgen-dependent hereditary disorder in which dihydrotestosterone plays\na major role.\nOther common causes of hair loss are\n• Drugs (including chemotherapeutic agents)\n• Infection\nThe Merck Manual of Diagnosis & Therapy, 19th Edition\nChapter 86. Hair Disorders\n846\njpkarthikk@outlook.com\nJS6E3YI1Z7\nThis file is meant for personal use by jpkarthikk@outlook.com only.\nSharing or publishing the contents in part or full is liable for legal action.',
  'metadata': {'Chunk_idx': 16, 'batch_id': 34},
  'similari

### Query 4 What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [73]:
rag_Query_04 = 'What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?'
result_rag_query_04 = Retrival_Response_From_RAG(rag_Query_04,vector_store)
result_rag_query_04

[{'content': 'brain damage is traumatic. Even if some recovery occurs after these intervals, most patients are severely\ndisabled. Rarely, improvement occurs late; after 5 yr, about 3% of patients recover the ability to\ncommunicate and comprehend, but even fewer can live independently; no patients regain normal\nfunction.\nMost patients in a persistent vegetative state die within 6 mo of the original brain damage. The cause is\nusually pulmonary infection, UTI, or multiple organ failure, or death may be sudden and of unknown\ncause. For most of the rest, life expectancy is about 2 to 5 yr; a few patients live for decades.\nTreatment\n• Supportive care\nThere is no specific treatment, but supportive care should include the following:\n• Preventing systemic complications due to immobilization (eg, pneumonia, UTI, thromboembolic disease)\n• Providing good nutrition\n• Preventing pressure ulcers\nThe Merck Manual of Diagnosis & Therapy, 19th Edition\nChapter 174. Coma & Impaired Conscious

### Query 5  What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [74]:
rag_Query_05 = 'What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?'
result_rag_query_05 = Retrival_Response_From_RAG(rag_Query_05,vector_store)
result_rag_query_05

[{'content': '(eg, meniscal tears, cartilaginous injuries). Arteriography may be necessary for suspected arterial injuries\n(eg, some popliteal artery injuries). Nerve conduction studies may be indicated for nerve injuries.\nTreatment\n• Treatment of life- or limb-threatening injuries\n• Splinting\n• Definitive treatment (eg, reduction) for certain injuries\n• Rest, ice, compression, and elevation (RICE)\n• Usually immobilization\nIn the emergency department, hemorrhagic shock is treated. Injuries to arteries are repaired surgically\nunless they affect only small arteries with good collateral circulation. Severed nerves are surgically\nrepaired; for neuropraxia and axonotmesis, initial treatment is usually observation, supportive measures,\nand sometimes physical therapy.\nMost injuries, particularly grossly unstable ones, are immobilized immediately by splinting (immobilization\nwith a nonrigid or noncircumferential device) to prevent further injury to soft tissues by unstable injurie

## Fine Tuning

In [112]:
def Response_Generated_Fine_tuned(query,chunk_retrived,max_new_tokens=256, temperature=0.7,top_p=0.9, top_k=50, USE_GPU=False):
  try:
    tokenizer = AutoTokenizer.from_pretrianed('distilgpt2')
    tokenizer.pad_token = tokenizer.eos_token
    generator = pipeline('text-generation', model='distilgpt2', tokenizer=tokenizer,
                        device=0 if USE_GPU and torch.cuda.is_available else -1)
    context = "\n".join([chunk_op['content'] for chunk_op in chunk_retrived])

    if not context:
      return "No Relevant information retrived from the model for the query asked"

    prompt = f"As per Merck Manual for Diagnostics: \n {context} \n Answer: {query}"

    if len(prompt) > 3000:
      prompt = prompt[:3000]

    response_generated = generator(prompt, max_new_tokens=max_new_tokens,
                          num_return_sequences=1,temperature =temperature,
                          top_p=top_p,top_k=top_k)[0]['generated_text']

    if "Answer:" in response_generated:
      ans = response_generated.split('Answer:')[1].strip()
    else:
      ans = response_generated.strip()

      print(f'Response Generated for the query asked {query}: \n Response \n {ans}')

    return ans

  except Exception as EX:
    print(f'Exception {EX}')


In [117]:
Response_Generated_Fine_tuned(rag_Query_01,result_rag_query_01,max_new_tokens=256,temperature=0.75,top_p=0.93,top_k=50)

In [114]:
def different_Combinations(query,chuk_retrival,vector_store,USE_GPU=False):

  generator = pipeline('text-generation', model='distilgpt2',device=0 if USE_GPU and torch.cuda.is_available else -1)
  combo = [
      {'k':3, 'similarity_threshold':0.0, 'max_new_tokens':256, 'temperature':0.75, 'top_p':0.93, 'top_k':50},
      {'k':5, 'similarity_threshold':0.2, 'max_new_tokens':128, 'temperature':0.77, 'top_p':0.95, 'top_k':50},
      {'k':2, 'similarity_threshold':0.4, 'max_new_tokens':150, 'temperature':0.85, 'top_p':0.90, 'top_k':40},
      {'k':1, 'similarity_threshold':0.1, 'max_new_tokens':100, 'temperature':0.9, 'top_p':0.85, 'top_k':50},
      {'k':4, 'similarity_threshold':0.5, 'max_new_tokens':300, 'temperature':0.95, 'top_p':0.80, 'top_k':47}

      ]

  for indx, parameters in enumerate(combo,1):
    print(f'k = {parameters["k"]}')
    print(f'similarity_threshold = {parameters["similarity_threshold"]}')
    print(f'max_new_tokens = {parameters["max_new_tokens"]}')
    print(f'temperature = {parameters["temperature"]}')
    print(f'top_p = {parameters["top_p"]}')
    print(f'top_k = {parameters["top_k"]}')

    #chunk_retrival = Retrival_Response_From_RAG(query=query,vector_store=vector_store,
     #                 k=parameters["k"],similarity_threshold=parameters["similarity_threshold"])

    response_gen = Response_Generated_Fine_tuned(rag_Query_01, chuk_retrival,
                                      max_new_tokens = parameters["max_new_tokens"],
                                      top_p = parameters["top_p"],
                                      top_k= parameters["top_k"],USE_GPU=True)

    print(f'Retrival_Response_From_RAG \n {chuk_retrival}')
    print('-'*200)
    print(f'Response_Generated \n {response_gen}')


In [116]:
different_Combinations(rag_Query_01,result_rag_query_01,vector_store)

Device set to use cpu


# Response Evaluation

In [111]:
QNA_System_Message = """You are the Medicak Assistant with expertise in provisioning the diagnostics based on The Merck Manuak. Provide an accurate and concise answers to medical questions using the given context from The Merck Manual"""

In [112]:
User_Message_Template = """Context: {context} Question: {question} """

In [113]:
groundedness_rater_system_message = """ You are tasked with evaluating the AI generated reponse for the user propmpted questions based on the groundness.
                                        Groundness measures how well the answer is supported by the providec context.
                                        Evaluate the answer based on the context and question, and provide the rating
                                        from the scale 1 to 5 where 1= not grounded, 5= fully grounded
                                        """

In [114]:
relevance_rater_system_message = """ You are tasked with evaluating the AI generated reponse for the user propmpted questions based on the relevance of a generated answer.
                                        Relevance measures how well the answer address the user question.
                                        Evaluate the answer based on the context and question, and provide the rating
                                        from the scale 1 to 5 where 1= not grounded, 5= fully grounded """

In [118]:
def generate_ground_relevance_response(user_input,vector_store,k=3,max_tokens=256,temperature=0.7,top_p=0.95,top_k=50,USE_GPU=False):
    #global qna_system_message,qna_user_message_template
    # Retrieve relevant document chunks
    relevant_document_chunks = Retrival(user_input,vector_store,k=k)
    context_list = [doc_chunk['content'] for doc_chunk in relevant_document_chunks]
    context_for_query = ". ".join(context_list)

    generator = pipeline('text-generation', model='distilgpt2', device=0 if USE_GPU and torch.cuda.is_available else -1)

    # Combine user_prompt and system_message to create the prompt
    prompt = f"""[INST]{QNA_System_Message}\n
                {'user'}: {QNA_System_Message.format(context=context_for_query,
                                                     question=user_input)}
                [/INST]"""

    if len(prompt) > 3000:
      prompt = prompt[:3000]

    response = generator(
            text_inputs=prompt,
            max_new_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            truncation = True
            )

    answer =  response[0]["generated_text"].split("[/INST]")[-1].strip()
    truncate_context = context_for_query[:3000]
    truncated_answer = answer[:3000]

    # Combine user_prompt and system_message to create the prompt
    evaluator = pipeline('text-classification', model ='distilgpt2',device =0 if USE_GPU and torch.cuda.is_available else -1)
    groundedness_prompt = f"""[INST]{groundedness_rater_system_message}\n
                {'user'}: {User_Message_Template.format(context=truncate_context, question=user_input, answer=truncated_answer)}
                [/INST]"""
    relevance_prompt = f"""[INST]{relevance_rater_system_message}\n
                {'user'}: {User_Message_Template.format(context=truncate_context, question=user_input, answer=truncated_answer)}
                [/INST]"""

    result_groundness = evaluator(groundedness_prompt)
    result_relevance = evaluator(relevance_prompt)

    #score_of_groundness = result_groundness

    return answer, result_groundness, result_relevance

In [119]:
Evaluation_query = 'What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?'